In [32]:
import _sqlite3
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import re
import datetime

import importlib
import common.lottery_results
importlib.reload(common.lottery_results)
from common.lottery_results import LotteryResultsCRUD

import inspect

pd.set_option('display.width', 4500)

In [33]:
sqlite_db_name = 'lottery_checker.sqlite'
connection = _sqlite3.connect(sqlite_db_name)
crud = LotteryResultsCRUD(sqlite_db_name)

method = crud.check_and_insert
num_args = len(inspect.signature(method).parameters)
print(f"The method requires {num_args} arguments.")

The method requires 1 arguments.


In [ ]:
query = "Select * from lottery_meta where active = 'yes'"

df = pd.read_sql_query(query, connection)

for index, meta in df.iterrows():
    
    # calculate the ending year, this applies to the Euro millions as changed format
    # build the year range
    if meta.lottery_end_year is None or (isinstance(meta.lottery_end_year, float) and math.isnan(meta.lottery_end_year)):
        end_year = datetime.datetime.now().year + 1
    else:
        end_year = math.floor(meta.lottery_end_year + 1)
    year_range = list(range(meta.lottery_start_year, end_year))

    lottery_slug = meta.lottery_slug
    lottery_id = meta.lottery_id
    lottery_name = meta.lottery_name
    lottery_class = meta.lottery_class
    
    col_string = meta.lottery_archive_columns
    cols = col_string.split('|')
    #cols = ['draw_date_str'] + cols
    #cols = ['lottery_id'] + cols
    #cols = ['draw_id'] + cols
    
    # loop over year_range to scrape year results
    for year in year_range:
               
        base_url = 'https://www.lottery.co.uk/{}/results/archive-{}'.format(lottery_slug, year)
        
        response = requests.get(base_url)
        html = response.content

        soup = BeautifulSoup(html, "html.parser")
        table = soup.find("table", class_=lottery_class)
        
        if table:
            
            rows = table.find_all("tr")
            
            for idx, row in enumerate(rows[1:], start=1):
                      
                cells = row.find_all(["td", "th"])
                row_data = [cell.text.strip() for cell in cells]
                
                # Extract the href value from the first cell (assuming it's a link)
                first_cell = cells[0]
                href_value = first_cell.find('a')['href']
                
                # Split the href value to extract the date part
                parts = href_value.split('-')
                date_str = '-'.join(parts[-1:] + parts[-3:-1])
                date_key = lottery_id + '|' + date_str
            
                # Add the extracted date as a new item in the dictionary
                data_dict = {key: value for key, value in zip(cols, row_data)}
                data_dict['lottery_id'] = lottery_id
                data_dict['draw_id'] = date_key
                data_dict['draw_date_str'] = date_str
                data_df = pd.DataFrame(columns=cols)
                
                # Append the dictionary to the DataFrame
                data_df = data_df.append(data_dict, ignore_index=True)
                
                # Melt the DataFrame while keeping 'draw_id' and 'lottery_id' columns
                melted_df = pd.melt(data_df, id_vars=['draw_id', 'lottery_id', 'draw_date_str'], var_name='draw_column', value_name='draw_values')
                        
                # check the db.lottery_results to see if [draw_id] exists
                # draw_id, draw_date, draw_column, draw_values, lottery_id
                crud.check_and_insert(melted_df)
                
                del melted_df
                del data_df
        

In [88]:
query = "select lottery_id, lottery_balls_draw from lottery_meta where active = 'yes'"

df = pd.read_sql_query(query, connection)

for index, meta in df.iterrows():
    
    lottery_id = meta.lottery_id

    lottery_balls_draw = meta.lottery_balls_draw
    lottery_balls_draw = lottery_balls_draw.split('|')
    
    print('lottery: {} - - - cols: {}'.format(lottery_id, lottery_balls_draw))
    
    query = "select draw_id, draw_values from lottery_results where draw_column = 'results' and lottery_id = 's4l'"
    
    draws = pd.read_sql_query(query, connection)
    
    for idx, draw in draws.iterrows():

        balls_clean =  draw.astype(str).replace('\n', ' ').str.split()
        print((balls_clean))
        
        # Split the input string by '|' to get 'main' and 'bonus'
        values = lottery_balls_draw.split('|')
        
        # Convert 'main' and 'bonus' to integers
        main_value = int(values[0])
        bonus_value = int(values[1])
        
        # Slice the list using the 'main_value' and 'bonus_value'
        main_list = balls_clean[:main_value]
        bonus_list = balls_clean[main_value:main_value+bonus_value]
        
        print(main_list)
        print(main_list)
        print('=-=-=-==-=-=-')

lottery: s4l - - - cols: ['5', '1']
draw_id              [s4l|2019-30-12]
draw_values    [8, 18, 21, 23, 44, 4]
Name: 0, dtype: object


AttributeError: 'list' object has no attribute 'split'

In [85]:
# Your input string and list
input_string = '5|2'
my_list = [23, 24, 25, 30, 31, 9, 18]

# Split the input string by '|' to get 'main' and 'bonus'
values = input_string.split('|')

# Convert 'main' and 'bonus' to integers
main_value = int(values[0])
bonus_value = int(values[1])

# Slice the list using the 'main_value' and 'bonus_value'
main_list = my_list[:main_value]
bonus_list = my_list[main_value:main_value+bonus_value]

# Print the results
print("Main:", main_list)
print("Bonus:", bonus_list)

Main: [23, 24, 25, 30, 31]
Bonus: [9, 18]
